# Generate Dyanmical Regression Matrix


Import Libraries


In [1]:
import sympy as p
import numpy as np
import sys
from SYMBOLIC import *

## 1. Kinematics with PoE


### 1.1 Variable Definitions


Define time $t$ and angles.


In [2]:
t = p.Symbol("t")

# Differentialable Variables
theta_1_t = p.Function("theta_1")(t)
theta_2_t = p.Function("theta_2")(t)

# Undifferentialable Variables
theta_1, theta_2 = p.symbols("theta_1 theta_2")
v_1, v_2 = p.symbols("v_1 v_2")
a_1, a_2 = p.symbols("a_1 a_2")

Define $\vec{\theta}$, $\dot{\vec{\theta}}$ and $\ddot{\vec{\theta}}$.


In [3]:
theta = p.Matrix([theta_1, theta_2]).reshape(2, 1)
theta_t = p.Matrix([theta_1_t, theta_2_t]).reshape(2, 1)


v = p.Matrix([v_1, v_2]).reshape(2, 1)
theta_dot = p.Matrix([theta_1_t.diff(t), theta_2_t.diff(t)]).reshape(2, 1)

a = p.Matrix([a_1, a_2]).reshape(2, 1)
theta_ddot = p.Matrix([theta_1_t.diff(t, 2), theta_2_t.diff(t, 2)]).reshape(2, 1)

Define robot parameters.


In [4]:
m1, m2, l1, l2, lc1, lc2, Jc1, Jc2, g = p.symbols("m1 m2 l1 l2 lc1 lc2 Jc1 Jc2 g")

# Jiang's Simulation
# l1 = 1.7
# l2 = 2.1
# g = 9.8

Define substitution functions from differentialable variables to undifferentialable variables.

In [5]:
def ThetaT2Theta(Expression):
    return (
        Expression.subs(theta_1_t.diff(t, 2), a_1)
        .subs(theta_2_t.diff(t, 2), a_2)
        .subs(theta_1_t.diff(t), v_1)
        .subs(theta_2_t.diff(t), v_2)
        .subs(theta_1_t, theta_1)
        .subs(theta_2_t, theta_2)
    )

Define geometric parameters. We apply **Modified D-H** notation. Noting that $\square_{i}$ refers to $\square$ of joint $i$, $\square_{ci}$ refers to $\square$ of rod $i$.


In [6]:
# Joint DH Parameters
x0, x1, x2 = p.symbols("x_0 x_1 x_2")
y0, y1, y2 = p.symbols("y_0 y_1 y_2")
z0, z1, z2 = p.symbols("z_0 z_1 z_2")

# Link CoM Parameters
xc1, xc2 = p.symbols("x_c1 x_c2")
yc1, yc2 = p.symbols("y_c1 y_c2")
zc1, zc2 = p.symbols("z_c1 z_c2")

From the configuration, we got

In [7]:
# Joint Parameters
x0, y0, z0 = 0, 0, 0
x1, y1, z1 = 0, 0, 0
x2, y2, z2 = l1, 0, 0

# CoM Parameters
xc1, yc1, zc1 = lc1, 0, 0
xc2, yc2, zc2 = lc2, 0, 0

# RPY Angles
RPY_0_1 = [0, 0, 0]
RPY_1_2 = [0, 0, 0]
RPY_1_c1 = [0, 0, 0]
RPY_2_c2 = [0, 0, 0]

### 1.2 Forward Kinematics


Transformation matrixes at $\vec{0}$. `T_i_j_0` refers to $T_{i}^{j} (\vec{0}) $.


In [8]:
T_1_0_0 = Translation_4x4(p.Matrix([x1, y1, z1])) @ Rotation_RPY_4x4(RPY_0_1)
T_2_1_0 = Translation_4x4(p.Matrix([x2, y2, z2])) @ Rotation_RPY_4x4(RPY_1_2)

Joint positions at $\vec{0}$.

In [9]:
def Position(Transformation_i_j, P_i=p.Matrix([0, 0, 0, 1])):
    return (Transformation_i_j @ P_i)[:3, :]


T_1_0_0 = T_1_0_0
T_2_0_0 = T_1_0_0 @ T_2_1_0

r_1_0 = Position(T_1_0_0)
r_2_0 = Position(T_2_0_0)

Joint poses at $\vec{0}$.

In [10]:
omega_1_0 = p.Matrix([0, 0, 1]).reshape(3, 1)
omega_2_0 = Rotation_RPY_4x4(RPY_1_2)[:3, :3] @ omega_1_0

Joint unitary twists at $\vec{0}$.

In [11]:
xi_1_0 = Joint2Twist(omega_1_0, r_1_0)
xi_2_0 = Joint2Twist(omega_2_0, r_2_0)

Transformation matrixes at $\vec{\theta}$ computed with PoE.


In [12]:
T_1_0 = MatrixExp_4x4(xi_1_0, theta_1_t) @ T_1_0_0
T_2_0 = MatrixExp_4x4(xi_1_0, theta_1_t) @ MatrixExp_4x4(xi_2_0, theta_2_t) @ T_2_0_0

Joint positions and poses at $\vec{\theta}$.

In [13]:
# Positions
r_1 = Position(T_1_0)
r_2 = Position(T_2_0)

# Poses
omega_1 = p.simplify(MatrixExp_3x3(omega_1_0, theta_1_t) @ omega_1_0)
omega_2 = p.simplify(
    MatrixExp_3x3(omega_1_0, theta_1_t)
    @ MatrixExp_3x3(omega_2_0, theta_2_t)
    @ omega_2_0
)

Spatial velocities at $\vec{\theta}$.


In [14]:
SpatialVelocity_Rot_1 = p.simplify(omega_1 * theta_1_t.diff(t))
SpatialVelocity_Rot_2 = p.simplify(
    omega_1 * theta_1_t.diff(t) + omega_2 * theta_2_t.diff(t)
)
SpatialVelocity_Trans_1 = p.simplify(p.simplify(r_1).diff(t))
SpatialVelocity_Trans_2 = p.simplify(p.simplify(r_2).diff(t))

SpatialVelocity_1 = SpatialVelocity_Rot_1.col_join(SpatialVelocity_Trans_1)
SpatialVelocity_2 = SpatialVelocity_Rot_2.col_join(SpatialVelocity_Trans_2)

## 2. Dynamical Regression Matrix


Dynamical parameter vectors $\vec{p}_{i}$.


In [15]:
p_1 = p.Matrix(
    [
        Jc1 / 2 + m1 * (yc1**2 + zc1**2),
        -(0 + m1 * xc1 * yc1),
        -(0 + m1 * xc1 * zc1),
        Jc1 / 2 + m1 * (xc1**2 + zc1**2),
        -(0 + m1 * yc1 * zc1),
        Jc1 + m1 * (xc1**2 + yc1**2),
        m1 * xc1,
        m1 * yc1,
        m1 * zc1,
        m1,
    ]
).reshape(10, 1)
p_2 = p.Matrix(
    [
        Jc2 / 2 + m2 * (yc2**2 + zc2**2),
        -(0 + m2 * xc2 * yc2),
        -(0 + m2 * xc2 * zc2),
        Jc2 / 2 + m2 * (xc2**2 + zc2**2),
        -(0 + m2 * yc2 * zc2),
        Jc2 + m2 * (xc2**2 + yc2**2),
        m2 * xc2,
        m2 * yc2,
        m2 * zc2,
        m2,
    ]
).reshape(10, 1)
p_vec = p_1.col_join(p_2)

$\tilde{T}^{i}$ s.


In [16]:
Ttilde_1 = p.simplify(Ttilde_10x1(SpatialVelocity_1, T_1_0))
Ttilde_2 = p.simplify(Ttilde_10x1(SpatialVelocity_2, T_2_0))

Verify $\tilde{T}^{i}$ s.


In [17]:
p.simplify(ThetaT2Theta(Ttilde_2.T @ p_2))

Matrix([[l1**2*m2*v_1**2/2 + l1*lc2*m2*v_1*(v_1 + v_2)*cos(theta_2) + (Jc2 + lc2**2*m2)*(v_1 + v_2)**2/2]])

$\tilde{V}^{i}$ s.


In [18]:
Vtilde_1 = p.simplify(Vtilde_10x1(r_1, p.Matrix([0, -g, 0]).reshape(3, 1), T_1_0))
Vtilde_2 = p.simplify(Vtilde_10x1(r_2, p.Matrix([0, -g, 0]).reshape(3, 1), T_2_0))

Verify $\tilde{V}^{i}$ s.


In [19]:
p.simplify(ThetaT2Theta(Vtilde_2.T @ p_2))

Matrix([[g*m2*(l1*sin(theta_1) + lc2*sin(theta_1 + theta_2))]])

$\tilde{T}$ and $\tilde{V}$.


In [20]:
Ttilde = Ttilde_1.col_join(Ttilde_2)
Vtilde = Vtilde_1.col_join(Vtilde_2)

Dynamical regression matrix $R_{\text{Full}}$.


In [21]:
R_Full = p.simplify(
    ThetaT2Theta(
        VecDiff(Ttilde, theta_dot).diff(t)
        - VecDiff(Ttilde, theta_t)
        + VecDiff(Vtilde, theta_t)
    )
)
R_Full

Matrix([
[0, 0, 0, 0, 0, a_1, g*cos(theta_1), -g*sin(theta_1), 0, 0, 0, 0, 0, 0, 0, a_1 + a_2, 2*a_1*l1*cos(theta_2) + a_2*l1*cos(theta_2) + g*cos(theta_1 + theta_2) - 2*l1*v_1*v_2*sin(theta_2) - l1*v_2**2*sin(theta_2), -2*a_1*l1*sin(theta_2) - a_2*l1*sin(theta_2) - g*sin(theta_1 + theta_2) - 2*l1*v_1*v_2*cos(theta_2) - l1*v_2**2*cos(theta_2), 0, l1*(a_1*l1 + g*cos(theta_1))],
[0, 0, 0, 0, 0,   0,              0,               0, 0, 0, 0, 0, 0, 0, 0, a_1 + a_2,                                                     a_1*l1*cos(theta_2) + g*cos(theta_1 + theta_2) + l1*v_1**2*sin(theta_2),                                                     -a_1*l1*sin(theta_2) - g*sin(theta_1 + theta_2) + l1*v_1**2*cos(theta_2), 0,                            0]])

In [22]:
print(p.latex(R_Full))

\left[\begin{array}{cccccccccccccccccccc}0 & 0 & 0 & 0 & 0 & a_{1} & g \cos{\left(\theta_{1} \right)} & - g \sin{\left(\theta_{1} \right)} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & a_{1} + a_{2} & 2 a_{1} l_{1} \cos{\left(\theta_{2} \right)} + a_{2} l_{1} \cos{\left(\theta_{2} \right)} + g \cos{\left(\theta_{1} + \theta_{2} \right)} - 2 l_{1} v_{1} v_{2} \sin{\left(\theta_{2} \right)} - l_{1} v_{2}^{2} \sin{\left(\theta_{2} \right)} & - 2 a_{1} l_{1} \sin{\left(\theta_{2} \right)} - a_{2} l_{1} \sin{\left(\theta_{2} \right)} - g \sin{\left(\theta_{1} + \theta_{2} \right)} - 2 l_{1} v_{1} v_{2} \cos{\left(\theta_{2} \right)} - l_{1} v_{2}^{2} \cos{\left(\theta_{2} \right)} & 0 & l_{1} \left(a_{1} l_{1} + g \cos{\left(\theta_{1} \right)}\right)\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & a_{1} + a_{2} & a_{1} l_{1} \cos{\left(\theta_{2} \right)} + g \cos{\left(\theta_{1} + \theta_{2} \right)} + l_{1} v_{1}^{2} \sin{\left(\theta_{2} \right)} & - a_{1} l_{1} \sin{\left(\theta_{2} \rig

Remove zero columns in $R_{\text{Full}}$ to get $R$.


In [23]:
cols = [
    i for i in range(R_Full.shape[1]) if R_Full[:, i] != p.zeros(R_Full.shape[0], 1)
]
R = R_Full[:, cols]
print("R's shape is", R.shape)
print("Non-zero columns are", [i + 1 for i in cols])
R

R's shape is (2, 7)
Non-zero columns are [6, 7, 8, 16, 17, 18, 20]


Matrix([
[a_1, g*cos(theta_1), -g*sin(theta_1), a_1 + a_2, 2*a_1*l1*cos(theta_2) + a_2*l1*cos(theta_2) + g*cos(theta_1 + theta_2) - 2*l1*v_1*v_2*sin(theta_2) - l1*v_2**2*sin(theta_2), -2*a_1*l1*sin(theta_2) - a_2*l1*sin(theta_2) - g*sin(theta_1 + theta_2) - 2*l1*v_1*v_2*cos(theta_2) - l1*v_2**2*cos(theta_2), l1*(a_1*l1 + g*cos(theta_1))],
[  0,              0,               0, a_1 + a_2,                                                     a_1*l1*cos(theta_2) + g*cos(theta_1 + theta_2) + l1*v_1**2*sin(theta_2),                                                     -a_1*l1*sin(theta_2) - g*sin(theta_1 + theta_2) + l1*v_1**2*cos(theta_2),                            0]])

## 3. Generate $M \left( \vec{q} \right)$, $C \left( \vec{q}, \dot{\vec{q}} \right)$, and $G \left( \vec{q} \right)$


Total kinematic and potential energy.

In [24]:
T = Ttilde.T @ p_vec
V = Vtilde.T @ p_vec

Lagrangian.

In [25]:
L = T - V
L

Matrix([[-g*l1*m2*sin(theta_1(t)) - g*lc1*m1*sin(theta_1(t)) - g*lc2*m2*sin(theta_1(t) + theta_2(t)) + l1**2*m2*Derivative(theta_1(t), t)**2/2 + l1*lc2*m2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*cos(theta_2(t))*Derivative(theta_1(t), t) + (Jc1 + lc1**2*m1)*Derivative(theta_1(t), t)**2/2 + (Jc2 + lc2**2*m2)*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))**2/2]])

$M$ matrix.

In [26]:
import itertools

temp = ThetaT2Theta(T.jacobian(theta_dot).diff(t)).subs(v_1, 0).subs(v_2, 0)

M = p.zeros(2, 2)
for i, j in itertools.product(range(2), range(2)):
    M[i, j] = temp[i].diff(a[j])

M

Matrix([
[Jc1 + Jc2 + l1**2*m2 + 2*l1*lc2*m2*cos(theta_2) + lc1**2*m1 + lc2**2*m2, Jc2 + l1*lc2*m2*cos(theta_2) + lc2**2*m2],
[                               Jc2 + l1*lc2*m2*cos(theta_2) + lc2**2*m2,                          Jc2 + lc2**2*m2]])

In [27]:
print(p.octave_code(M[1, 1]))

Jc2 + lc2.^2.*m2


$C$ matrix.

In [28]:
C = p.zeros(2, 2)
for i, j, k in itertools.product(range(2), range(2), range(2)):
    C[i, j] += (
        v[k]
        / 2
        * (M[i, k].diff(theta[j]) + M[i, j].diff(theta[k]) - M[j, k].diff(theta[i]))
    )

C

Matrix([
[-l1*lc2*m2*v_2*sin(theta_2), -l1*lc2*m2*v_1*sin(theta_2) - l1*lc2*m2*v_2*sin(theta_2)],
[ l1*lc2*m2*v_1*sin(theta_2),                                                        0]])

In [49]:
CVec = p.simplify(C @ v)

In [51]:
print(p.octave_code(CVec[1]))

l1.*lc2.*m2.*v_1.^2.*sin(theta_2)


$G$ vector.

In [29]:
G = ThetaT2Theta(V).jacobian(theta).T
G

Matrix([
[g*l1*m2*cos(theta_1) + g*lc1*m1*cos(theta_1) + g*lc2*m2*cos(theta_1 + theta_2)],
[                                               g*lc2*m2*cos(theta_1 + theta_2)]])

In [42]:
print(p.octave_code(G[1]))

g.*lc2.*m2.*cos(theta_1 + theta_2)


Check $R_{\text{Full}}$ correctness.

In [30]:
p.simplify(M @ a + C @ v + G - R_Full @ p_vec)

Matrix([
[0],
[0]])

## 4. Generate $Y \left( \vec{q}, \dot{\vec{q}}, \vec{\phi}, \vec{\psi} \right)$

Define $\vec{\phi}$ and $\vec{\psi}$.

In [31]:
phi_1 = p.Symbol("phi_1")
phi_2 = p.Symbol("phi_2")
psi_1 = p.Symbol("psi_1")
psi_2 = p.Symbol("psi_2")
phi = p.Matrix([phi_1, phi_2]).reshape(2, 1)
psi = p.Matrix([psi_1, psi_2]).reshape(2, 1)

Substitute $\vec{v}$ and $\vec{a}$ relevant terms with $\vec{\phi}$ and $\vec{\psi}$ included terms

In [32]:
Y_Full = RMat2YMat(R_Full, v, a, phi, psi)
Y = RMat2YMat(R, v, a, phi, psi)

$Y_{\text{Full}}$

In [33]:
Y_Full

Matrix([
[0, 0, 0, 0, 0, psi_1, g*cos(theta_1), -g*sin(theta_1), 0, 0, 0, 0, 0, 0, 0, psi_1 + psi_2, g*cos(theta_1 + theta_2) - 1.0*l1*phi_2*v_2*sin(theta_2) + 2*l1*psi_1*cos(theta_2) + l1*psi_2*cos(theta_2) - 2*l1*(0.5*phi_1*v_2 + 0.5*phi_2*v_1)*sin(theta_2), -g*sin(theta_1 + theta_2) - 1.0*l1*phi_2*v_2*cos(theta_2) - 2*l1*psi_1*sin(theta_2) - l1*psi_2*sin(theta_2) - 2*l1*(0.5*phi_1*v_2 + 0.5*phi_2*v_1)*cos(theta_2), 0, l1*(g*cos(theta_1) + l1*psi_1)],
[0, 0, 0, 0, 0,     0,              0,               0, 0, 0, 0, 0, 0, 0, 0, psi_1 + psi_2,                                                                               g*cos(theta_1 + theta_2) + 1.0*l1*phi_1*v_1*sin(theta_2) + l1*psi_1*cos(theta_2),                                                                               -g*sin(theta_1 + theta_2) + 1.0*l1*phi_1*v_1*cos(theta_2) - l1*psi_1*sin(theta_2), 0,                              0]])

Verify the result.

In [34]:
p.simplify(M @ psi + C @ phi + G - Y_Full @ p_vec)

Matrix([
[0],
[0]])

$Y$.


In [35]:
print("Y's shape is", Y.shape)
Y

Y's shape is (2, 7)


Matrix([
[psi_1, g*cos(theta_1), -g*sin(theta_1), psi_1 + psi_2, g*cos(theta_1 + theta_2) - 1.0*l1*phi_2*v_2*sin(theta_2) + 2*l1*psi_1*cos(theta_2) + l1*psi_2*cos(theta_2) - 2*l1*(0.5*phi_1*v_2 + 0.5*phi_2*v_1)*sin(theta_2), -g*sin(theta_1 + theta_2) - 1.0*l1*phi_2*v_2*cos(theta_2) - 2*l1*psi_1*sin(theta_2) - l1*psi_2*sin(theta_2) - 2*l1*(0.5*phi_1*v_2 + 0.5*phi_2*v_1)*cos(theta_2), l1*(g*cos(theta_1) + l1*psi_1)],
[    0,              0,               0, psi_1 + psi_2,                                                                               g*cos(theta_1 + theta_2) + 1.0*l1*phi_1*v_1*sin(theta_2) + l1*psi_1*cos(theta_2),                                                                               -g*sin(theta_1 + theta_2) + 1.0*l1*phi_1*v_1*cos(theta_2) - l1*psi_1*sin(theta_2),                              0]])

Dynamical regression vector $\vec{y}$.


In [36]:
y = Y.reshape(Y.shape[0] * Y.shape[1], 1)
y

Matrix([
[                                                                                                                                                          psi_1],
[                                                                                                                                                 g*cos(theta_1)],
[                                                                                                                                                -g*sin(theta_1)],
[                                                                                                                                                  psi_1 + psi_2],
[ g*cos(theta_1 + theta_2) - 1.0*l1*phi_2*v_2*sin(theta_2) + 2*l1*psi_1*cos(theta_2) + l1*psi_2*cos(theta_2) - 2*l1*(0.5*phi_1*v_2 + 0.5*phi_2*v_1)*sin(theta_2)],
[-g*sin(theta_1 + theta_2) - 1.0*l1*phi_2*v_2*cos(theta_2) - 2*l1*psi_1*sin(theta_2) - l1*psi_2*sin(theta_2) - 2*l1*(0.5*phi_1*v_2 + 0.5*phi_2*v_1)*cos(theta_2)],
[            

Jiang's manual simplification.

In [37]:
Y[:, [0, 1, 3, 4, 6]]

Matrix([
[psi_1, g*cos(theta_1), psi_1 + psi_2, g*cos(theta_1 + theta_2) - 1.0*l1*phi_2*v_2*sin(theta_2) + 2*l1*psi_1*cos(theta_2) + l1*psi_2*cos(theta_2) - 2*l1*(0.5*phi_1*v_2 + 0.5*phi_2*v_1)*sin(theta_2), l1*(g*cos(theta_1) + l1*psi_1)],
[    0,              0, psi_1 + psi_2,                                                                               g*cos(theta_1 + theta_2) + 1.0*l1*phi_1*v_1*sin(theta_2) + l1*psi_1*cos(theta_2),                              0]])

Corresponding to following elements in $\vec{y}$.

In [38]:
y[[0, 1, 3, 4, 6, 7, 8, 10, 11, 13], :] == Y[:, [0, 1, 3, 4, 6]].reshape(10, 1)

True

In [39]:
print(p.latex(y[[0, 1, 3, 4, 6, 7, 8, 10, 11, 13], :]))

\left[\begin{matrix}\psi_{1}\\g \cos{\left(\theta_{1} \right)}\\\psi_{1} + \psi_{2}\\g \cos{\left(\theta_{1} + \theta_{2} \right)} - 1.0 l_{1} \phi_{2} v_{2} \sin{\left(\theta_{2} \right)} + 2 l_{1} \psi_{1} \cos{\left(\theta_{2} \right)} + l_{1} \psi_{2} \cos{\left(\theta_{2} \right)} - 2 l_{1} \left(0.5 \phi_{1} v_{2} + 0.5 \phi_{2} v_{1}\right) \sin{\left(\theta_{2} \right)}\\l_{1} \left(g \cos{\left(\theta_{1} \right)} + l_{1} \psi_{1}\right)\\0\\0\\\psi_{1} + \psi_{2}\\g \cos{\left(\theta_{1} + \theta_{2} \right)} + 1.0 l_{1} \phi_{1} v_{1} \sin{\left(\theta_{2} \right)} + l_{1} \psi_{1} \cos{\left(\theta_{2} \right)}\\0\end{matrix}\right]


Check reshaping.

In [40]:
y.reshape(2, 7) == Y

True